In [0]:
# This comment is here to prevent notebook from disconnecting...
# %% [code]
# TODO:
# Implement save parameter in train().

# %% [code]
RANDOM_SEED = 44

# %% [code]
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import os
import gc
import time
import warnings
warnings.filterwarnings('ignore')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

from functools import partial
from bayes_opt import BayesianOptimization, SequentialDomainReductionTransformer
from bayes_opt.logger import JSONLogger, ScreenLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# %% [code]
class StackingClassifier(object):
    def __init__(self, classifiers, base_classifier, nfolds):
        self.classifiers = classifiers
        self.nclassifiers = len(classifiers)
        self.base_classifier = base_classifier
        self.nfolds = nfolds
        self._kf = KFold(n_splits=nfolds, shuffle=True, random_state=RANDOM_SEED)
        
    def train(self, xtrain, ytrain):
        base_xtrn = np.zeros(shape=(xtrain.shape[0], self.nclassifiers))                               
        for iclf, clf in enumerate(self.classifiers):
            print(f'\nTraining {type(clf).__name__} model.')
            base_xtrn[:, iclf] = self._train_clf(clf, xtrain, ytrain)

        print('\nTraining base classifier...')
        self.base_classifier.train(base_xtrn, ytrain)
        print('Done.')
                                      
    def _train_clf(self, clf, xtrain, ytrain):
        val_preds = np.zeros(shape=(xtrain.shape[0],))
        for ifold, (trn_idx, val_idx) in enumerate(self._kf.split(xtrain)):
            print(f'--Fold {ifold + 1}...')
            xtrn = xtrain.loc[trn_idx]
            ytrn = ytrain.loc[trn_idx]
            xval = xtrain.loc[val_idx]
            yval = ytrain.loc[val_idx]
            
            clf.train(xtrn, ytrn)
            clf_preds = clf.predict(xval)
            val_preds[val_idx] = clf_preds
            print("    	validation's auc", metrics.roc_auc_score(yval, clf_preds))
            
        print('--Training on all data...')
        clf.train(xtrain, ytrain)
        return val_preds
    
    def predict(self, xtest):
        base_xtest = np.zeros(shape=(xtest.shape[0], self.nclassifiers))
        for iclf, clf in enumerate(self.classifiers):
            base_xtest[:, iclf] = clf.predict(xtest)
        return self.base_classifier.predict(base_xtest)
    
    def score(self, xtest, ytest):
        preds = self.predict(xtest)
        return metrics.roc_auc_score(ytest, preds)

# %% [code]
class LGBMClassifier(object):
    def __init__(self, params):
        self.params = params
        self.params['feature_fraction_seed'] = RANDOM_SEED
        self.params['bagging_seed'] = RANDOM_SEED
        self.log = {}
        self.clf = None
        self.best_score = 0
        
    def train(self, train, valid, verbose=1):
        train = lgb.Dataset(train['x'], label=train['y'], free_raw_data=True)
        valid = lgb.Dataset(valid['x'], label=valid['y'], free_raw_data=True, reference=train)
        self.log = {}
        self.clf = lgb.train(self.params, 
                             train,
                             valid_sets=[valid],
                             callbacks=[lgb.record_evaluation(self.log),
                                        lgb.early_stopping(12, verbose=verbose)], 
                             verbose_eval=verbose)
        self.best_score = max(self.log['valid_0'][opt_params['metric']])
        
    def predict(self, xtest):
        return self.clf.predict(xtest)

# %% [code]
class XGBClassifier(object):
    def __init__(self, params):
        self.params = params
        self.params['seed'] = RANDOM_SEED
        self.nrounds = params.pop('nrounds', 100)
        self.clf = None
        self.log = {}
        self.best_score = 0
        
    def train(self, train, valid, verbose=1):
        train = xgb.DMatrix(train['x'], label=train['y'])
        valid = xgb.DMatrix(valid['x'], label=valid['y'])
        self.log = {}
        self.clf = xgb.train(self.params, 
                             train, 
                             evals=[(valid, 'valid')],
                             num_boost_round=self.nrounds,
                             early_stopping_rounds=12,
                             verbose_eval=verbose)
        self.best_score = self.clf.best_score
        
    def predict(self, xtest):
        return self.clf.predict(xgb.DMatrix(xtest))

# %% [code]
class CATBClassifier(object):
    def __init__(self, params):
        self.params = params
        self.params['random_seed'] = RANDOM_SEED       
        self.clf = None
        self.best_score = 0

    def train(self, train, valid, verbose=1):
        self.clf = CatBoostClassifier(verbose=verbose, **self.params)
        self.clf.fit(train['x'], train['y'],
                     eval_set=(valid['x'], valid['y']),
                     verbose=verbose,
                     use_best_model=True,
                     early_stopping_rounds=12)
        self.best_score = self.clf.get_best_score()['validation'][self.params['eval_metric']]

    def predict(self, xtest):
        return self.clf.predict_proba(xtest)[:, 1]

# %% [code]
class SklearnClassifier(object):
    def __init__(self, params):
        self.params = params
        self.params['random_state'] = RANDOM_SEED
        self.clf_type = self.params.pop('clf_type')
        self.clf = None
        self.best_score = 0
        
    def train(self, train, valid, verbose=0):
        self.clf = self.clf_type(**self.params, verbose=verbose)
        self.clf.fit(train['x'], train['y'])
        self.best_score = self.clf.score(valid['x'], valid['y'])
        
    def predict(self, xtest):
        return self.clf.predict_proba(xtest)[:, 1]

# %% [code]
class NeuralNetworkClassifier(object):
    def __init__(self, params):
        self.params = params
        self.clf = None
        self.epochs = self.params.pop('epochs', 5)
        self.batch_size = self.params.pop('batch_size', 5)
        self.log = {}
    def train(self, train, valid, verbose=2):
        self.clf = self._create_ann(**self.params)
        callback_es = EarlyStopping(monitor=f"val_{self.params['metric']}", 
                                    patience=10, mode='max', verbose=verbose)
        self.log = self.clf.fit(train['x'], train['y'],
                                epochs=self.epochs,
                                batch_size=self.batch_size,
                                validation_data=(valid['x'], valid['y']),
                                callbacks=[callback_es],
                                shuffle=True,
                                verbose=verbose)

    def predict(self, xtest):
        return self.clf.predict(xtest)
    
    def _create_ann(in_shape,
                    out_shape,
                    metric,
                    n_layers=1,
                    last_units=64,
                    units_decay=1,
                    hidden_sizes=None,
                    learning_rate=0.001, 
                    lr_decay=0.0,
                    dropout_rate=0.1,
                    norm=0):
        """
        Creates a multilayer perceptron model.

        :param in_shape: The shape of the input data.
        :param out_shape: The shape of the model output.
        :param n_layers: The number of hidden dense layers.
        :param last_units: The number of units for the last hidden dense layer.
        :param units_decay: The decay rate of the number of units going from last hidden layer to the first.
        :param hidden_sizes: A list containing the number of nodes for each hidden dense layer. This is meant to be used as 
            an alternative to n_layers, n_units, and units_scale to specify the number of hidden layers and the number of 
            nodes per hidden layer.
        :param learning_rate: The learning rate of the model.
        :param lr_decay: The learning rate decay of the model.
        :param dropout_rate: The dropout rate for the dropout layers.
        :param norm: Whether to use a batch normalization layer after each dense layer or not.

        :return An MLP model.
        """
        kb.clear_session()
        tf.compat.v1.reset_default_graph()

        n_layers = int(np.floor(n_layers))
        last_units = int(last_units)
        norm = int(round(norm))
        model = Sequential()

        # Adding the input and hidden layers.
        if hidden_sizes is not None:
            for i_layer, i_units in enumerate(hidden_sizes):
                if i_layer == 0:
                    model.add(Dense(i_units, activation='relu',
                                    input_shape=in_shape))
                else:
                    model.add(Dense(i_units, activation='relu'))                
                if norm:
                    model.add(BatchNormalization())
                if dropout_rate:
                    model.add(Dropout(dropout_rate)) 
        else:
            for i_layer in range(n_layers):
                if i_layer == 0:
                    model.add(Dense(max(1, int(last_units*(units_decay**(n_layers-i_layer-1)))), 
                                    activation='relu', 
                                    input_shape=in_shape))
                else:
                    model.add(Dense(max(1, int(last_units*(units_decay**(n_layers-i_layer-1)))), 
                                    activation='relu'))                
                if norm:
                    model.add(BatchNormalization())
                if dropout_rate:
                    model.add(Dropout(dropout_rate)) 

        # Adding the output layer
        model.add(Dense(units=out_shape, activation='sigmoid'))

        model.compile(optimizer=optimizers.Adam(lr=learning_rate, decay=lr_decay), 
                      loss="binary_crossentropy", metrics=metric)

        return model

# %% [code]
def get_scorer(clf, train, valid, fixed_params, opt_dtypes):
    return partial(scorer, clf, train, valid, fixed_params, opt_dtypes)

def scorer(clf, train, valid, fixed_params, opt_dtypes, **opt_params):
    for param in opt_params:
        opt_params[param] = opt_dtypes[param](opt_params[param])
    opt_params.update(fixed_params)
    model = clf(opt_params)
    model.train(train, valid, verbose=0)

    return model.best_score

# %% [code]
class Timer(object):
    def __init__(self, engine):
        engine.subscribe(Events.OPTIMIZATION_START, self)
        engine.subscribe(Events.OPTIMIZATION_STEP, self)
        self.init_time = None

    def update(self, event, instance):
        if event == Events.OPTIMIZATION_START:
            self.init_time = time.time()
        else:
            time_taken = time.time() - self.init_time
            print("Time Taken:", time.strftime('%H:%M:%S', time.gmtime(time_taken)), end='\r')
            self.init_time = time.time()

def get_optimizer(scorer, params_grid, filename, cont_opt=False, probes=[], use_transformer=False):
    transformer = SequentialDomainReductionTransformer(gamma_osc=0.8, eta=0.95)
    optimizer = BayesianOptimization(f=scorer,
                                     pbounds=params_grid,
                                     verbose=2,
                                     bounds_transformer=transformer if use_transformer else None)

    scr_logger = ScreenLogger()
    scr_logger._default_cell_size = 10
    optimizer.subscribe(Events.OPTIMIZATION_START, scr_logger)
    optimizer.subscribe(Events.OPTIMIZATION_STEP, scr_logger)
    optimizer.subscribe(Events.OPTIMIZATION_END, scr_logger)    
    
    if cont_opt:
        if os.path.exists(filename):
            optimizer.dispatch(Events.OPTIMIZATION_START)
            optimizer.unsubscribe(Events.OPTIMIZATION_START, scr_logger)
            load_logs(optimizer, logs=[filename])
        else:
            print('File not found:', filename)
    else:
        for ps in probes:
            optimizer.probe(lazy=True, params=ps)     
        
    f_logger = JSONLogger(path=filename, reset=False if cont_opt else True)
    optimizer.subscribe(Events.OPTIMIZATION_STEP, f_logger)    
    Timer(optimizer)

    return optimizer